In [1]:
import polars as pl 
import numpy as np
from sklearn.datasets import make_classification
from sklearn.feature_selection import f_regression
import pandas as pd
from time import perf_counter
import sys
sys.path.append('../src')
from eda.eda_selection import _f_score, mrmr, MRMR_STRATEGY

In [2]:
# create some data
orig_x, orig_y = make_classification(n_samples = 500_000, n_features = 200, n_informative = 20, n_redundant = 180)

In [3]:
df_pl = pl.from_numpy(orig_x).insert_at_idx(0, pl.Series("target", orig_y))
df_pd = df_pl.to_pandas()

In [4]:
def mrmr_my_rewrite(df:pl.DataFrame, target:str, k:int) -> list[str]:

    features = df.columns
    features.remove(target)

    start = perf_counter()
    f_scores = _f_score(df, target, features)
    end_1 = perf_counter()
    print(f"Spent {end_1 - start:.2f}s to compute f score.")

    df_scaled = df.select(features).with_columns(
        (pl.col(f) - pl.col(f).mean())/pl.col(f).std() for f in features
    )

    cumulating_sum = np.zeros(len(features)) # For each feature at index i, we keep a cumulating sum
    top_idx = np.argmax(f_scores)
    selected_features = [features[top_idx]]
    for j in range(1, k): 
        argmax = -1
        current_max = -1
        last_selected = selected_features[-1]
        for i,f in enumerate(features):
            if f not in selected_features:
                # Left = cumulating sum of abs corr
                # Right = abs correlation btw last_selected and f
                cumulating_sum[i] += np.abs((df_scaled.get_column(last_selected)*df_scaled.get_column(f)).mean())
                denominator = cumulating_sum[i] / j
                new_score = f_scores[i] / denominator
                if new_score > current_max:
                    current_max = new_score
                    argmax = i

        selected_features.append(features[argmax])

    end_2 = perf_counter()
    print(f"Took {end_2 - end_1:.2f}s to compute mrmr.")
    return selected_features
    

In [5]:
def mrmr_np_rewrite(df:pl.DataFrame, target:str, k:int) -> list[str]:

    features = df.columns
    features.remove(target)

    start = perf_counter()
    f_scores = _f_score(df, target, features)
    end_1 = perf_counter()
    print(f"Spent {end_1 - start:.2f}s to compute f score.")

    x = df.select(features).to_numpy()
    x_scaled = (x - x.mean(axis=0)) / x.std(axis=0)

    cumulating_sum = np.zeros(len(features)) # For each feature at index i, we keep a cumulating sum
    top_idx = np.argmax(f_scores)
    selected_features = [features[top_idx]]
    last_selected_idx = top_idx
    for j in range(1, k): 
        argmax = -1
        current_max = -1
        for i,f in enumerate(features):
            if f not in selected_features:
                # Left = cumulating sum of abs corr
                # Right = abs correlation btw last_selected and f
                cumulating_sum[i] += np.abs(np.mean(x_scaled[:,last_selected_idx]*x_scaled[:,i]))
                denominator = cumulating_sum[i] / j
                new_score = f_scores[i] / denominator
                if new_score > current_max:
                    current_max = new_score
                    argmax = i

        selected_features.append(features[argmax])
        last_selected_idx = argmax

    end_2 = perf_counter()
    print(f"Took {end_2 - end_1:.2f}s to compute mrmr.")
    return selected_features

In [6]:
def mrmr_medium(df:pd.DataFrame, target:str, k:int) -> list[str]:

    features = list(df.columns)
    features.remove(target)

    X = df[features]
    y = df[target]

    # compute F-statistics and initialize correlation matrix
    start = perf_counter()
    F = pd.Series(f_regression(X, y)[0], index = X.columns)
    end_1 = perf_counter()
    print(f"Spent {end_1 - start:.2f}s to compute f score.")

    corr = pd.DataFrame(.00001, index = X.columns, columns = X.columns)

    # initialize list of selected features and list of excluded features
    selected = []
    not_selected = X.columns.to_list()

    # repeat K times
    for i in range(k):
    
        # compute (absolute) correlations between the last selected feature and all the (currently) excluded features
        if i > 0:
            last_selected = selected[-1]
            corr.loc[not_selected, last_selected] = X[not_selected].corrwith(X[last_selected]).abs().clip(.00001)
            
        # compute FCQ score for all the (currently) excluded features (this is Formula 2)
        score = F.loc[not_selected] / corr.loc[not_selected, selected].mean(axis = 1).fillna(.00001)
        
        # find best feature, add it to selected and remove it from not_selected
        best = score.index[score.argmax()]
        selected.append(best)
        not_selected.remove(best)

    # print(selected) 
    end_2 = perf_counter()
    print(f"Took {end_2 - end_1:.2f}s to compute mrmr.")
    return selected 

In [7]:
mrmr_my_rewrite(df_pl, "target", 50)

Spent 0.11s to compute f score.
Took 2.77s to compute mrmr.


['column_114',
 'column_149',
 'column_64',
 'column_188',
 'column_9',
 'column_44',
 'column_25',
 'column_48',
 'column_57',
 'column_5',
 'column_168',
 'column_53',
 'column_139',
 'column_197',
 'column_126',
 'column_81',
 'column_28',
 'column_147',
 'column_78',
 'column_40',
 'column_99',
 'column_191',
 'column_135',
 'column_33',
 'column_185',
 'column_196',
 'column_10',
 'column_85',
 'column_63',
 'column_186',
 'column_12',
 'column_164',
 'column_124',
 'column_143',
 'column_88',
 'column_60',
 'column_39',
 'column_52',
 'column_180',
 'column_79',
 'column_178',
 'column_169',
 'column_49',
 'column_127',
 'column_161',
 'column_163',
 'column_82',
 'column_118',
 'column_87',
 'column_14']

In [8]:
# Use my version of f_score computation.
# Use pure NumPy for the rest of the computation.
# This shows that copying to NumPy, does not pay off. Stay in Polars is a better idea.
mrmr_np_rewrite(df_pl, "target", 50)

Spent 0.12s to compute f score.
Took 9.72s to compute mrmr.


['column_114',
 'column_149',
 'column_64',
 'column_188',
 'column_9',
 'column_44',
 'column_25',
 'column_48',
 'column_57',
 'column_5',
 'column_168',
 'column_53',
 'column_139',
 'column_197',
 'column_126',
 'column_81',
 'column_28',
 'column_147',
 'column_78',
 'column_40',
 'column_99',
 'column_191',
 'column_135',
 'column_33',
 'column_185',
 'column_196',
 'column_10',
 'column_85',
 'column_63',
 'column_186',
 'column_12',
 'column_164',
 'column_124',
 'column_143',
 'column_88',
 'column_60',
 'column_39',
 'column_52',
 'column_180',
 'column_79',
 'column_178',
 'column_169',
 'column_49',
 'column_127',
 'column_161',
 'column_163',
 'column_82',
 'column_118',
 'column_87',
 'column_14']

In [9]:
mrmr_medium(df_pd, "target", 50)

Spent 0.27s to compute f score.
Took 44.94s to compute mrmr.


['column_114',
 'column_149',
 'column_64',
 'column_188',
 'column_9',
 'column_44',
 'column_25',
 'column_48',
 'column_57',
 'column_5',
 'column_168',
 'column_53',
 'column_139',
 'column_197',
 'column_126',
 'column_81',
 'column_28',
 'column_147',
 'column_78',
 'column_40',
 'column_99',
 'column_191',
 'column_135',
 'column_33',
 'column_185',
 'column_196',
 'column_10',
 'column_85',
 'column_63',
 'column_186',
 'column_12',
 'column_164',
 'column_124',
 'column_143',
 'column_88',
 'column_60',
 'column_39',
 'column_52',
 'column_180',
 'column_79',
 'column_178',
 'column_169',
 'column_49',
 'column_127',
 'column_161',
 'column_163',
 'column_82',
 'column_118',
 'column_87',
 'column_14']

In [10]:
# My yet to be published package. 2.8s
mrmr(df_pl, target="target", k = 50)

Top 5 feature importance is (by MRMR_STRATEGY.F_SCORE):
shape: (5, 2)
┌────────────┬───────────────────────┐
│ feature    ┆ MRMR_STRATEGY.F_SCORE │
│ ---        ┆ ---                   │
│ str        ┆ f64                   │
╞════════════╪═══════════════════════╡
│ column_114 ┆ 172456.754119         │
│ column_25  ┆ 149138.929177         │
│ column_9   ┆ 142909.371455         │
│ column_57  ┆ 138151.830291         │
│ column_5   ┆ 124816.909713         │
└────────────┴───────────────────────┘
Found 200 total features to select from. Proceeding to select top 50 features.


100%|██████████| 50/50 [00:02<00:00, 19.52it/s]


mrmr_rank,feature
i64,str
1,"""column_114"""
2,"""column_149"""
3,"""column_64"""
4,"""column_188"""
5,"""column_9"""
6,"""column_44"""
7,"""column_25"""
8,"""column_48"""
9,"""column_57"""


In [11]:
from mrmr import mrmr_classif

In [12]:
#
def mrmr_package(df:pd.DataFrame, target:str, k:int) -> list[str]:
    
    features = list(df.columns)
    features.remove(target)

    X = df[features]
    y = df[target]
    start = perf_counter()
    output = mrmr_classif(X, y, K = k)
    end = perf_counter()
    print(f"Spent {end - start:.2f}s to compute mrmr.")
    return output

In [13]:
mrmr_package(df_pd, "target", 50)

100%|██████████| 50/50 [01:13<00:00,  1.46s/it]

Spent 78.13s to compute mrmr.


['column_114',
 'column_69',
 'column_64',
 'column_57',
 'column_5',
 'column_9',
 'column_44',
 'column_25',
 'column_48',
 'column_168',
 'column_197',
 'column_188',
 'column_53',
 'column_139',
 'column_126',
 'column_78',
 'column_81',
 'column_147',
 'column_28',
 'column_135',
 'column_40',
 'column_99',
 'column_191',
 'column_196',
 'column_33',
 'column_185',
 'column_186',
 'column_63',
 'column_10',
 'column_85',
 'column_12',
 'column_124',
 'column_164',
 'column_88',
 'column_143',
 'column_39',
 'column_60',
 'column_180',
 'column_52',
 'column_79',
 'column_178',
 'column_169',
 'column_49',
 'column_127',
 'column_161',
 'column_82',
 'column_163',
 'column_118',
 'column_87',
 'column_14']

In [14]:
mrmr(df_pl, target="target", k=20, strategy=MRMR_STRATEGY.RF, params={"n_estimators":20, "max_depth":5, "n_jobs":-1})

Random forest is not deterministic by default. Results may vary.
Top 5 feature importance is (by MRMR_STRATEGY.RF):
shape: (5, 2)
┌────────────┬──────────────────┐
│ feature    ┆ MRMR_STRATEGY.RF │
│ ---        ┆ ---              │
│ str        ┆ f64              │
╞════════════╪══════════════════╡
│ column_114 ┆ 0.128501         │
│ column_9   ┆ 0.063352         │
│ column_57  ┆ 0.058116         │
│ column_139 ┆ 0.042677         │
│ column_188 ┆ 0.033205         │
└────────────┴──────────────────┘
Found 200 total features to select from. Proceeding to select top 20 features.


100%|██████████| 20/20 [00:00<00:00, 20.53it/s]


mrmr_rank,feature
i64,str
1,"""column_114"""
2,"""column_149"""
3,"""column_125"""
4,"""column_139"""
5,"""column_57"""
6,"""column_9"""
7,"""column_188"""
8,"""column_63"""
9,"""column_60"""


In [15]:
mrmr(df_pl, target="target", k=20, strategy=MRMR_STRATEGY.XGB, params={"n_estimators":20, "max_depth":5})

XGB is not deterministic by default. Results may vary.
Top 5 feature importance is (by MRMR_STRATEGY.XGB):
shape: (5, 2)
┌────────────┬───────────────────┐
│ feature    ┆ MRMR_STRATEGY.XGB │
│ ---        ┆ ---               │
│ str        ┆ f64               │
╞════════════╪═══════════════════╡
│ column_114 ┆ 0.080585          │
│ column_9   ┆ 0.069619          │
│ column_44  ┆ 0.051323          │
│ column_57  ┆ 0.037718          │
│ column_161 ┆ 0.03685           │
└────────────┴───────────────────┘
Found 200 total features to select from. Proceeding to select top 20 features.


100%|██████████| 20/20 [00:01<00:00, 19.04it/s]


mrmr_rank,feature
i64,str
1,"""column_114"""
2,"""column_167"""
3,"""column_9"""
4,"""column_44"""
5,"""column_161"""
6,"""column_57"""
7,"""column_125"""
8,"""column_143"""
9,"""column_25"""
